Twoim zadaniem jest wytrenowanie klasyfikatora binarnego na podzbiorze zbioru MNIST, w którym wyróżniamy klasy (cyfry 0 i 1 mają zostać wyłączone ze zbioru):
 - Liczby pierwsze (2,3,5,7)
 - Liczby złożone (4,6,8,9)

Napisz wydajną implementację modelu **regresji logistycznej** trenowanego algorytmem ***SGD z momentum***. Cały proces trenowania musisz napisać samodzielnie, w języku Python, korzystając z biblioteki numpy. Na potrzeby zadania niedozwolone jest korzystanie z gotowych implementacji optimizerów i modeli oraz bibliotek do automatycznego różniczkowania funkcji (np. Tensorflow, pytorch, autograd). 

Dobierz hiperparametry tak, aby uzyskać jak najlepszy wynik na zbiorze walidacyjnym. 
Wyciągnij i zapisz wnioski z przeprowadzonych eksperymentów.

Zbiór MNIST dostępny jest pod linkami: 

(zbiór treningowy):
 - http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
 - http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz

(zbiór walidacyjny):
 - http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
 - http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz



# Kod

In [1]:
import numpy as np
import idx2numpy
from math import exp
indexl=[]
indexv=[]

class Model:
  def __init__(self):
    self.tl= idx2numpy.convert_from_file('data/train-labels.idx1-ubyte')
    self.ti= idx2numpy.convert_from_file('data/train-images.idx3-ubyte')  
    self.vl= idx2numpy.convert_from_file('data/t10k-labels.idx1-ubyte')  
    self.vi= idx2numpy.convert_from_file('data/t10k-images.idx3-ubyte')
    self.tl.setflags(write=1)
    self.ti.setflags(write=1)
    self.vl.setflags(write=1)
    self.vi.setflags(write=1)

  def fit(self) -> None:
    for x in range(len(self.tl)):
        if self.tl[x]==4 or self.tl[x]==6 or self.tl[x]==8 or self.tl[x]==9:
            self.tl[x]=0
        elif self.tl[x]==2 or self.tl[x]==3 or self.tl[x]==5 or self.tl[x]==7:
            self.tl[x]=1
        elif self.tl[x]==0 or self.tl[x]==1:
            indexl.append(x)

    for x in range(len(self.vl)):
        if self.vl[x]==4 or self.vl[x]==6 or self.vl[x]==8 or self.vl[x]==9:
            self.vl[x]=0
        elif self.vl[x]==2 or self.vl[x]==3 or self.vl[x]==5 or self.vl[x]==7:
            self.vl[x]=1
        elif self.vl[x]==0 or self.vl[x]==1:
            indexv.append(x)
    self.tl=np.delete(self.tl,indexl)
    self.ti=np.delete(self.ti,indexl,axis=0)
    self.vl=np.delete(self.vl,indexv)
    self.vi=np.delete(self.vi,indexv,axis=0)
    self.vi=self.vi.astype(float)
    for x in range(len(self.vl)):
        for y in range(28):
            for z in range(28):
                nor=self.vi[x][y][z]/255.0
                self.vi[x][y][z]=nor
    self.ti=self.ti.astype(float)
    for x in range(len(self.tl)):
        for y in range(28):
            for z in range(28):
                nor=self.ti[x][y][z]/255.0
                self.ti[x][y][z]=nor
                
    self.ti=self.ti.T
    self.ti=self.ti.reshape(-1,self.ti.shape[-1])
    self.ti=self.ti.T
    self.vi=self.vi.T
    self.vi=self.vi.reshape(-1,self.vi.shape[-1])
    self.vi=self.vi.T

  def calc(self, data, wsp):
    pred = wsp[0]
    for i in range(len(data)):
        pred += wsp[i + 1] * data[i]
    return 1.0 / (1.0 + exp(-pred))

  def sgd(self, train, check , rate, n):
    wsp = np.zeros(785)
    for i in range(n):
        for x in range(len(train)):
            pred = self.calc(train[x], wsp)
            var = check[x] - pred
            wsp[0] = wsp[0] + rate * var * pred * (1.0 - pred)
            for j in range(len(train[x])):
                wsp[j + 1] = wsp[j+1] + rate * var * pred * (1.0 - pred) * train[x][j]
        print(i)
    return wsp

  def predict(self, arr, check, wsp):
        y_pred=np.zeros(len(check))
        for x in range(len(arr)):
            pred=round(self.calc(arr[x],wsp))
            y_pred[x]=pred
        print(self.evaluate(self.vl, y_pred))

  @staticmethod
  def evaluate(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    g = 0
    for x in range(len(y_true)):
        if y_true[x]==y_pred[x]:
            g+=1
    return g/len(y_true)


In [2]:
model=Model()


In [3]:
model.fit()

In [25]:
rate = 0.5
n = 50
wsp = model.sgd(model.ti[0:10000], model.tl, rate, n)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [26]:
#print(wsp)

In [27]:
model.predict(model.vi,model.vl,wsp)

0.9135066582117946


# Opis
Niestety nie udało mi się zaimplementować SGD z momentum. To moja próba użycia regresji logistycznej trenowanej algorytmem SGD. Poniżej przedstawiam wyniki kilku testów. Sposób przedstawienia: n - liczba pętli, rate - ograniczenie korekty współcznnika, acc - wynik przetestowania modelu na zbiorze testowym, learning - tam gdzie został ograniczony zbiór uczący parametr learning pokazuje jego liczebność. Mała ilość pętli oraz ograniczenia elementów zbioru uczącego zostały spowodowane ograniczeniami sprzętowymi.

* n: 1 rate: 0.5                  acc: 0.91224
* n: 1 rate: 0.2                  acc: 0.91122
* n: 1 rate: 1.0                  acc: 0.90641
* n: 1 rate: 0.3                  acc: 0.90919
* n: 5 rate: 0.5 learning: 10000  acc: 0.90717
* n: 5 rate: 0.3 learning: 10000  acc: 0.91325
* n: 20 rate: 0.5 learning: 10000 acc: 0.91325
* n: 20 rate: 0.3 learning: 10000 acc: 0.91262
* n: 50 rate: 0.5 learning: 10000 acc: 0.91351






# Wnioski

* Długi czas wykonania programu może być nie tylko spowodowany niskimi możliwościami obliczeniowymi sprzętu na którym przeprowadzono testy ale też błędnej/mało wydajnej implementacji algorytmu.

* Warto zaznaczyć że dane na których były przeprowadzane testy zostały znormalizowane i szczególnie atrybuty ti i vi zostały zmienione z tablic 3-wymiarowych na 2-wymiarowe. Może to być nieoptymale i powodować spadek dokładności modelu.

* Można zauwżyć że najwyższa dokładność jest dla n=50 co oznacza, że ilość iteracji pętli wyliczającej współczynniki tablicy w modelu zwiększa dokładność. Warto jednak zwrócić uwagę, że 5 iteracji daje słabsze wyniki niż tylko jedna iteracja, i że 20 iteracji ma lepszy wynik niż 1 i 5 iteracji ale mniej niż 50.

* Występuje różnica pomiędzy ograniczeniem korekcji współczynników. Najleiej wypada 0.5, a najgorzej 1.

* Należało by dodatkowo sprawdzić skuteczność innych parametrów rate przy większej liczbie iteracji niż tylko 0.5 i 0.3.

* Przy większej liczbie iteracji rate=0.3 wypada podobnie do rate-0.5 dla 20 iteracji, ale lepiej przy 5 iteracjach.

* Aby poprawić dokładność należało by zastanowić się nad sposobem przygotowania danych i być może go poprawić.

* Warto zwrócić uwagę na fakt, że dla większej liczby iteracji niż 1 został użyty zmniejszony zbiór uczący. Należało by na lepszym sprzęcie przeprowadzić testy bez pomniejszonego zbioru uczącego.